In [3]:
import speech_recognition as sr

In [4]:
import pandas as pd
df=pd.read_csv("question_set.csv",encoding='ISO-8859-1')
df

,sr_no,question,ans,ans1
0,1.0,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...","Python is a widely-used general-purpose, high-..."
1,2.0,What are the benefits of using Python language...,Object-Oriented Language\nHigh-Level Language\...,Object-Oriented Language\nHigh-Level Language\...
2,3.0,What is the difference between a Mutable datat...,"Mutable data types can be edited i.e., they ca...","Mutable data types can be edited i.e., they ca..."
3,4.0,What is the difference between a Set and Dicti...,The set is an unordered collection of data typ...,The set is an unordered collection of data typ...
4,5.0,What is a pass in Python?,Pass means performing no operation or in other...,Pass means performing no operation or in other...
5,6.0,Difference between for loop and while loop in ...,The for Loop is generally used to iterate th...,The for Loop is generally used to iterate th...
6,NaN,NaN,NaN,NaN


In [5]:
df.isnull().sum()

sr_no       1
question    1
ans         1
ans1        1
dtype: int64

In [6]:
df.fillna("Not Mentioned",inplace=True)

C:\Users\dipse\AppData\Local\Temp\ipykernel_3488\285520692.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Mentioned' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Not Mentioned",inplace=True)


In [7]:
df.isnull().sum()

sr_no       0
question    0
ans         0
ans1        0
dtype: int64

In [11]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [12]:
def preprocess_text(text):
    text=text.lower()
    words=word_tokenize(text)
    stop_word=set(stopwords.words('english'))
    words=[i for i in words if i not in stop_word]
    stemmer=PorterStemmer()
    words=[stemmer.stem(i) for i in words]
    preprocessed_text=' '.join(words)
    return preprocessed_text

In [ ]:
s="I am Dipseek working as Data Scientist." #testing above fun
preprocess_text(s)

'dipseek work data scientist .'

In [14]:
df['ans1']=df['ans1'].apply(preprocess_text)
df.head(1)

,sr_no,question,ans,ans1
0,1.0,What is Python? List some popular applications...,"Python is a widely-used general-purpose, high-...","python widely-us general-purpos , high-level p..."


In [15]:
import pickle
with open('preprocess_text.pkl','wb')as file:
    pickle.dump(df,file)

In [17]:
df['question']=df['question'].str.strip() #strip() removes unwanted symbols like /

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
def suggest_section_for_selected_question(ans,df,question_index):

    #preprocess the user's ans
    preprocessed_ans=preprocess_text(ans)
    ans_embedding=model.encode(preprocessed_ans)

    #get the actual ans for the selected question
    actual_ans=df.iloc[question_index]['ans']
    preprocessed_actual_ans=preprocess_text(actual_ans)
    actual_ans_embedding=model.encode(preprocessed_actual_ans)

    #compute the similarity between user's ans and actual ans
    similarity_score=util.pytorch_cos_sim(ans_embedding,actual_ans_embedding)[0][0].item()

    return{
        'question': df.iloc[question_index]['question'],
        'actual_ans':actual_ans,
        'similarity_score':similarity_score
    }

#display the list of questions
print("Following are the questions:\n")
for idx, question in enumerate(df['question'],1):
    print(f"{idx}. {question}")

#ask the user to choose the question
n=int(input("Which question number you want to answer?"))

#get the user's ans
ans=input("Enter your answer: ")

#get the suggestion for the selected question
suggestion=suggest_section_for_selected_question(ans,df,n-1)

#display the result
if suggestion:
    print(f"\n Question Number: {n}")
    print(f"Question: {suggestion['question']}")
    print(f"Actual Ans: {suggestion['actual_ans']}")
    print(f"\n Your Ans: {ans}")
    print(f"{round(suggestion['similarity_score']*100)}% your answer is correct.")
      


Following are the questions:

1. What is Python? List some popular applications of Python in the world of technology.
2. What are the benefits of using Python language as a tool in the present scenario?
3. What is the difference between a Mutable datatype and an Immutable data type?
4. What is the difference between a Set and Dictionary?
5. What is a pass in Python?
6. Difference between for loop and while loop in Python
7. Not Mentioned

 Question Number: 1
Question: What is Python? List some popular applications of Python in the world of technology.
Actual Ans: Python is a widely-used general-purpose, high-level programming language. It was created by Guido van Rossum in 1991 and further developed by the Python Software Foundation. It was designed with an emphasis on code readability, and its syntax allows programmers to express their concepts in fewer lines of code.

 Your Ans: python is high lever programming language
55% your answer is correct.


In [26]:
import application as ap
ap.audio_to_text("C:/Users/dipse/OneDrive/Desktop/data science/5th sem/Speech To Text Project/temp.wav")

'for loop is used to perform the iteration whereas while loop does not perform the iteration for loop is used with list tuple dictionary whereas we cannot use the while loop on list set and dictionary'